In [1]:
import os
from binascii import hexlify
from time import sleep
import datetime

In [2]:
import tagcore

In [3]:
import tagnet

TagNet Driver Version 0.2.1


In [33]:
tagdir       = '/home/pi/tags/1fbcd99fd29f'
dblkdir      = tagdir+'/tag/sd/0/dblk' 
dblkbyte     = dblkdir+'/byte' 
dblkresync   = dblkdir+'/.resync'
dblklastsync = dblkdir+'/.last_sync'
dblkrecnum   = dblkdir+'/.recnum'
dblkcommitted= dblkdir+'/.committed'

In [5]:
!pwd

/mnt/neptune/TagNet/notebooks


In [6]:
def int32(x):
  if x>0xFFFFFFFF:
    raise OverflowError
  if x>0x7FFFFFFF:
    x=int(0x100000000-x)
    if x<2147483648:
      return -x
    else:
      return -2147483648
  return x

In [7]:
def round_to_word(a):
    return ((a-(a%4)) if (a%4) else (a))

In [8]:
def get_file_size(fn):
    ret=!stat  -c '%s' $fn
    offset=int(ret[0])
    int32(offset)
    return offset

In [9]:
def err_by_value(val):
    for e in tagnet.tlv_errors:
        if e.value == val: break
    return e.name

In [10]:
def size_to_err(pos):
    ipos = int32(pos)
    if ipos < 0:
        print('error', -ipos, err_by_value(-ipos))
        return -ipos
    return ipos

In [11]:
def resync(fname, offset):
    !truncate -s $offset $fname
    limit = 1
    while (limit < 100):
        offset = get_file_size(dblkresync)
        if (offset):
            #print('=', limit, offset)
            return offset, limit
        limit += 1
        #print('.', limit, offset)
    return -1, 0

In [12]:
def record_check(fn, offset):
    os.lseek(fn, offset, os.SEEK_SET)
    hdr=tagcore.core_headers.obj_dt_hdr()
    buf=bytearray(os.read(fn, len(hdr)))
    if len(buf) < len(hdr): return 0
    hdr.set(buf)
    rlen=hdr['len'].val
    rtype  = hdr['type'].val
    recnum = hdr['recnum'].val
    recsum = hdr['recsum'].val
    rt = hdr['rt']
    dlen = rlen - len(hdr)
    if (dlen > 0):
        buf.extend(os.read(fn, dlen))
    chksum = sum(bytearray(buf)[:rlen])
    chksum -= (recsum & 0xff00) >> 8
    chksum -= (recsum & 0x00ff)
    chksum &= 0xffff                # force to 16 bits vs. 16 bit recsum
    if (chksum != recsum):
        print('checksum error, expected: {:x}, got: {:x}'.format(chksum, recsum))
        print('.'.join(['{:02x}'.format(a) for a in buf[:20]]))
    else:
        dt = datetime.datetime(rt['year'].val, rt['mon'].val, rt['day'].val, rt['hr'].val, rt['min'].val, rt['sec'].val, rt['sub_sec'].val)
        print('{}: {}, len: {}, offset: {}, recnum: {}, recsum: {:x}/{:x}'.format(tagcore.dt_defs.dt_name(rtype),dt.isoformat(),rlen,offset,recnum,recsum,chksum))
        print('.'.join(['{:02x}'.format(a) for a in buf]))
    return round_to_word(rlen)

In [13]:
def sync_check(fn, offset, display=False):
    os.lseek(fn, offset, os.SEEK_SET)
    sync=tagcore.core_headers.obj_dt_sync()
    buf=bytearray(os.read(fn, len(sync)))
    sync.set(buf)
    rlen   = sync['hdr']['len'].val
    rtype  = sync['hdr']['type'].val
    recnum = sync['hdr']['recnum'].val
    recsum = sync['hdr']['recsum'].val
    rt = sync['hdr']['rt']
    dt = datetime.datetime(rt['year'].val, rt['mon'].val, rt['day'].val, rt['hr'].val, rt['min'].val, rt['sec'].val, rt['sub_sec'].val)
    prev_sync = sync['prev_sync'].val
    dlen = rlen - len(sync)
    if (dlen > 0):
        buf.extend(os.read(fn, dlen))
    chksum = sum(bytearray(buf)[:rlen])
    chksum -= (recsum & 0xff00) >> 8
    chksum -= (recsum & 0x00ff)
    chksum &= 0xffff                # force to 16 bits vs. 16 bit recsum
    if (display):
        print('{}: {}, len: {}, offset/prev: {}/{}, recnum: {}, recsum: {:x}/{:x}'.format(
            tagcore.dt_defs.dt_name(rtype),
            dt.isoformat(),
            rlen,
            offset,
            prev_sync,
            recnum,
            recsum,
            chksum))
        print('.'.join(['{:02x}'.format(a) for a in buf]))
    if (chksum != recsum):
        print('checksum error, expected: {:x}, got: {:x}'.format(chksum, recsum))
    return prev_sync

# Dblk Info

In [14]:
dblk_fn=os.open(dblkbyte, os.O_RDWR)

In [15]:
!ls -al $dblkbyte

-r--r--r-- 1 pi pi 908192 Nov 13 12:39 /home/pi/tags/1fbcd99fd29f/tag/sd/0/dblk/byte


In [16]:
!ls -al $dblkdir/.recnum

-rw-rw-r-- 1 pi pi 22387 Nov 13 12:39 /home/pi/tags/1fbcd99fd29f/tag/sd/0/dblk/.recnum


In [17]:
!ls -al $dblkrecnum

-rw-rw-r-- 1 pi pi 22387 Nov 13 12:39 /home/pi/tags/1fbcd99fd29f/tag/sd/0/dblk/.recnum


In [18]:
!ls -al $dblkdir/.last_sync

-r--r--r-- 1 pi pi 907604 Nov 13 12:39 /home/pi/tags/1fbcd99fd29f/tag/sd/0/dblk/.last_sync


In [19]:
ret=!stat  -c '%s' $dblkdir/.last_sync
sync_offset=int(ret[0])
sync_offset

907604

In [20]:
record_check(dblk_fn, sync_offset)

dt/3: 2018-11-13T20:32:45.015195, len: 28, offset: 907604, recnum: 22373, recsum: 761/761
1c.00.03.8c.65.57.00.00.5b.3b.2d.20.14.01.0d.0b.e2.07.61.07.78.d0.0d.00.ef.00.df.de


28

In [21]:
sync_check(dblk_fn, sync_offset)

905336

# Find sync record

In [23]:
!tree -p -s -D -a $dblkdir

/home/pi/tags/1fbcd99fd29f/tag/sd/0/dblk
├── [-r--r--r--      908352 Nov 13 12:39]  byte
├── [-r--r--r--      907776 Nov 13 12:39]  .committed
├── [drwxr-x--x           0 Dec 31  1969]  filter
│   ├── [drwxr-x--x           0 Dec 31  1969]  exclude
│   │   └── [drwxr-x--x           0 Dec 31  1969]  DT_EVENT
│   └── [drwxr-x--x           0 Dec 31  1969]  include
│       └── [drwxr-x--x           0 Dec 31  1969]  DT_EVENT
├── [-r--r--r--      908312 Nov 13 12:39]  .last_rec
├── [-r--r--r--      907604 Nov 13 12:39]  .last_sync
├── [-rw-rw----           0 Nov 13 12:39]  note
├── [-r--r--r--           0 Dec 31  1969]  .offset
├── [-rw-rw-r--       22391 Nov 13 12:39]  .recnum
├── [-rw-rw-r--  4294967282 Nov 13 12:39]  .resync
└── [-r--r--r--           0 Dec 31  1969]  .size

5 directories, 9 files


In [24]:
!truncate -s 190976 $dblkdir/.resync

In [25]:
!truncate -s $sync_offset $dblkdir/.resync
!ls -al $dblkdir/.resync

-rw-rw-r-- 1 pi pi 907604 Nov 13 12:39 /home/pi/tags/1fbcd99fd29f/tag/sd/0/dblk/.resync


In [26]:
!ls -al $dblkdir/.resync

-rw-rw-r-- 1 pi pi 907604 Nov 13 12:39 /home/pi/tags/1fbcd99fd29f/tag/sd/0/dblk/.resync


In [27]:
resync_offset = get_file_size(dblkresync)

In [28]:
record_check(dblk_fn, resync_offset)

dt/3: 2018-11-13T20:32:45.015195, len: 28, offset: 907604, recnum: 22373, recsum: 761/761
1c.00.03.8c.65.57.00.00.5b.3b.2d.20.14.01.0d.0b.e2.07.61.07.78.d0.0d.00.ef.00.df.de


28

In [29]:
sync_check(dblk_fn, resync_offset)

905336

# Verify consecutive sync records

In [37]:
offset = get_file_size(dblkcommitted)
offset

907776

In [39]:
#offset=get_file_size(dblkresync)
last_offset = 0
start=datetime.datetime.now()
print(start)
for i in range(2000):
    rtime = datetime.datetime.now()
    offset, limit = resync(dblkresync, offset)
    ttime = datetime.datetime.now() - rtime
    offset = int32(offset)
    if offset <= 0: break
    try:
        previous = sync_check(dblk_fn, offset)
        delta = offset - previous if () else 0
        print('*** {:4}[{:2}], {:4}bps, {:8} : {}'.format(i,
                                                          limit,
                                                          round((delta*8)/ttime.total_seconds(),2),
                                                          offset, previous))
        if (i) and (previous != last_offset): print('missed sync, previous: {}, last: {}'.format(previous, last_offset))
    except OSError:
        if offset: print(offset)
        sleep(1)
    last_offset = offset
    offset += 4
    #print('loop',offset)
print('total {} seconds, {} iterations, eof: {}, resync: {}, error: {}'.format(
                                                        round((datetime.datetime.now()-start).total_seconds(),1),
                                                        i,
                                                        get_file_size(dblkbyte),
                                                        get_file_size(dblkresync),
                                                        err_by_value(-offset)))

2018-11-13 12:41:59.574971
total 2.9 seconds, 0 iterations, eof: 909552, resync: 4294967282, error: EODATA


In [47]:
get_file_size(dblklastsync), get_file_size(dblkbyte), get_file_size(dblkrecnum)

(912316, 912464, 22494)

In [48]:
offset = get_file_size(dblklastsync)
#offset = 460340

In [49]:
for i in range(100):
    if offset >= get_file_size(dblkbyte):
        sleep(1)
        print('offset: {}, eof: {}, last_sync: {}'.format(offset, get_file_size(dblkbyte), get_file_size(dblklastsync)))
        continue
    try:
        offset += record_check(dblk_fn, offset)
        print('*** count: {}, offset: {}, mod512: {}'.format(i, offset, offset % 512))
    except OSError:
        sleep(1)
        print('offset: {}, eof: {}, last_sync: {}'.format(offset, get_file_size(dblkbyte), get_file_size(dblklastsync)))
print('done')

dt/3: 2018-11-13T20:42:45.015195, len: 28, offset: 912316, recnum: 22491, recsum: 804/804
1c.00.03.e5.db.57.00.00.5b.3b.2d.2a.14.01.0d.0b.e2.07.04.08.30.e2.0d.00.ef.00.df.de
*** count: 0, offset: 912344, mod512: 472
dt/4: 2018-11-13T20:42:47.024334, len: 40, offset: 912344, recnum: 22492, recsum: 44e/44e
28.00.04.ca.dc.57.00.00.0e.5f.2f.2a.14.01.0d.0b.e2.07.4e.04.44.00.00.00.00.00.00.00.01.00.00.00.00.00.00.00.04.00.00.00
*** count: 1, offset: 912384, mod512: 0
dt/4: 2018-11-13T20:42:53.024296, len: 40, offset: 912384, recnum: 22493, recsum: 4a5/4a5
28.00.04.41.dd.57.00.00.e8.5e.35.2a.14.01.0d.0b.e2.07.a5.04.44.00.00.00.00.00.00.00.01.00.00.00.00.00.00.00.04.00.00.00
*** count: 2, offset: 912424, mod512: 40
dt/4: 2018-11-13T20:42:58.025283, len: 40, offset: 912424, recnum: 22494, recsum: 536/536
28.00.04.f0.de.57.00.00.c3.62.3a.2a.14.01.0d.0b.e2.07.36.05.42.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.04.00.00.00
*** count: 3, offset: 912464, mod512: 80
dt/4: 2018-11-13T20:43:05.024377

KeyboardInterrupt: 

In [ ]:
offset = get_file_size(dblklastsync)
while (True):
    offset = sync_check(dblk_fn, offset, display=True)    

# Other

In [ ]:
STOP

In [ ]:
i=4294967295
int32(i)

In [ ]:
int32()

In [ ]:
!ls

In [ ]:
help(tagcore)

In [ ]:
help(tagcore.core_headers)

In [ ]:
!ls /tmp